<a href="https://colab.research.google.com/github/mriitian/Reports/blob/main/AE_667_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Goals of Assignment 1
- Initiating the development of performance estimator tool and mission planner, with physics of hover and climb modes, and provision for future inclusion of other flight modes. (team effort)
- First-cut design of helicopter rotor(s) necessary for hover and climb modes using the developed tools. (individual effort)

## Teammates:
- Sahil R Patil
- Pritesh Dhakate
- Vedant Ganesh Parkhe
- Malavath Akshara Naik


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Define constants
R = 287.05             # J/(kg*K) - gas constant for air
gamma = 1.4            # ratio of specific heats
g = 9.81               # m/s^2
T0 = 288.15            # K - ISA sea level temp
p0 = 101325            # Pa - ISA sea level pressure
rho0 = 1.225           # kg/m3 - ISA sea level density
a0 = 340.3             # m/s - speed of sound at sea level

In [ ]:
# -------------------------------
# Part 2: Atmosphere Model
# -------------------------------
def atmosphere(altitude):
    """
    Computes temperature, pressure, density, and speed of sound
    at a given altitude using ISA model (up to ~11 km).

    Inputs:
        altitude (m)
    Returns:
        T (K), p (Pa), rho (kg/m3), a (m/s)
    """
    # Troposphere (up to 11 km)
    L = -0.0065  # temperature lapse rate (K/m)
    if altitude <= 11000:
        T = T0 + L * altitude
        p = p0 * (T / T0) ** (-g / (L * R))
    else:
        raise ValueError("Model only valid up to 11 km (for now).")

    rho = p / (R * T)
    a = np.sqrt(gamma * R * T)
    return T, p, rho, a

In [ ]:
# -------------------------------
# Part 3: Airfoil Model
# -------------------------------
def airfoil_coeff(alpha_rad):
    """
    Returns Cl and Cd for the airfoil used in Knight & Hefner (1937).

    Inputs:
        alpha_rad : angle of attack in radians

    Returns:
        Cl, Cd
    """
    alpha_deg = np.degrees(alpha_rad)
    Cl = 5.75 * alpha_rad  # slope given in per rad
    Cd = 0.0113 + 1.25 * (alpha_rad ** 2)
    return Cl, Cd

In [ ]:

# -------------------------------
# Part 4: Quick Test
# -------------------------------

# Atmosphere check at sea level and 2000 m
for h in [0, 2000]:
    T, p, rho, a = atmosphere(h)
    print(f"Altitude: {h} m | T={T:.2f} K | p={p:.1f} Pa | rho={rho:.3f} kg/m3 | a={a:.1f} m/s")

# Airfoil check for small angles of attack
alphas = np.radians(np.linspace(-10, 15, 6))  # -10° to +15°
for alpha in alphas:
    Cl, Cd = airfoil_coeff(alpha)
    print(f"alpha={np.degrees(alpha):.1f} deg | Cl={Cl:.3f} | Cd={Cd:.4f}")

In [ ]:
# -------------------------------
# Blade Geometry (linear taper + twist)
# -------------------------------
def blade_geometry(r, R, c_root, c_tip, theta_root, theta_tip, r_cutout=0.1):
    if r < r_cutout:
        return 0, 0
    c = c_root + (c_tip - c_root) * (r - r_cutout) / (R - r_cutout)
    theta = theta_root + (theta_tip - theta_root) * (r - r_cutout) / (R - r_cutout)
    return c, theta

In [ ]:
# -------------------------------
# Prandtl Tip Loss Factor
# -------------------------------
def prandtl_tip_loss(B, r, R, phi):
    if abs(np.sin(phi)) < 1e-6:   # avoid division by zero
        return 1.0
    f = (B/2) * (R - r) / (r * np.sin(phi))
    val = np.exp(-f)
    val = np.clip(val, 0.0, 1.0)  # ensure inside [0,1]
    F = (2/np.pi) * np.arccos(val)
    return max(F, 1e-3)  # avoid F = 0


In [ ]:
def bemt_section(r, R, b, Omega, rho, climb_vel,
                 c, theta, max_iter=100, tol=1e-5, debug=False):
    """
    Solve induced velocity and sectional forces at a given radius using BEMT (iterative BET–MT matching).
    """
    lambd = 0.1  # initial guess

    for i in range(max_iter):
        UT = Omega * r
        UP = climb_vel + lambd * Omega * R
        phi = np.arctan2(UP, UT)

        # AoA (sign convention fix)
        alpha = theta + phi
        Cl, Cd = airfoil_coeff(alpha)

        U = np.sqrt(UT**2 + UP**2)

        # BET forces
        dL = 0.5 * rho * U**2 * c * Cl
        dD = 0.5 * rho * U**2 * c * Cd
        dT_bet = dL * np.cos(phi) - dD * np.sin(phi)
        dQ_bet = r * (dD * np.cos(phi) + dL * np.sin(phi))

        # Momentum theory thrust
        F = max(prandtl_tip_loss(b, r, R, phi), 1e-3)
        v = lambd * Omega * R
        dT_mt = 4 * np.pi * r * rho * (climb_vel + v) * v * F

        # Update λ with relaxation
        if dT_mt > 1e-6:
            lambd_new = lambd + 0.1 * ((dT_bet - dT_mt) / dT_mt)
        else:
            lambd_new = lambd

        # Debug print
        # print(f"iter={i}, λ={lambd:.4f}, φ={np.degrees(phi):.2f}, dT_BET={dT_bet:.2f}, dT_MT={dT_mt:.2f}")

        # Convergence check
        if abs(lambd_new - lambd) < tol:
            lambd = lambd_new
            break
        lambd = lambd_new

    return dT_bet, dQ_bet, phi, lambd


In [ ]:
r = 0.3     # m (example radial station)
R = 0.762   # m (rotor radius)
b = 3       # blades
Omega = 2500 * 2*np.pi / 60  # rad/s
rho = 1.225
climb_vel = 0.0
c = 0.05    # chord
theta = np.radians(10)  # pitch

dT, dQ, phi, lambd = bemt_section(r, R, b, Omega, rho, climb_vel, c, theta)
print("Sectional thrust =", dT, "N/m")
print("Sectional torque =", dQ, "Nm/m")
print("Inflow angle φ =", np.degrees(phi), "deg")
print("Lambda =", lambd)


Sectional thrust = 365.3131901791488 N/m
Sectional torque = 26.877629952225945 Nm/m
Inflow angle φ = 9.26592507149323 deg
Lambda = 0.01241065012651952
